In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np

### Suppress warnings for now

In [2]:
import warnings
warnings.filterwarnings("ignore")

### Get SST and SSH Data

In [3]:
fSSH = 'http://apdrc.soest.hawaii.edu:80/dods/public_data/Reanalysis_Data/ORAS5/1x1_grid/sossheig/opa0_ext'
dSSH = xr.open_dataset(fSSH,decode_times=True)

fTEMP = 'http://apdrc.soest.hawaii.edu:80/dods/public_data/Reanalysis_Data/ORAS5/1x1_grid/votemper/opa0_ext'
dTEMP = xr.open_dataset(fTEMP,decode_times=True)

<xarray.Dataset>
Dimensions:   (lat: 180, lev: 75, lon: 360, time: 732)
Coordinates:
  * time      (time) datetime64[ns] 1958-01-15 1958-02-15 ... 2018-12-15
  * lev       (lev) float64 0.5058 1.556 2.668 ... 5.495e+03 5.698e+03 5.902e+03
  * lat       (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * lon       (lon) float64 0.0 1.0 2.0 3.0 4.0 ... 356.0 357.0 358.0 359.0
Data variables:
    votemper  (time, lev, lat, lon) float32 ...
Attributes:
    title:          ECMWF ocean reanalysis ORAS5 1x1-degree Potential Tempera...
    Conventions:    COARDS\nGrADS
    dataType:       Grid
    documentation:  http://apdrc.soest.hawaii.edu/datadoc/ecmwf_oras5_1x1.php
    history:        Fri Jun 05 09:23:11 HST 2020 : imported by GrADS Data Ser...

### Make anomalies and Nino3.4

In [ ]:
nino_total = dTEMP.votemper[:,0,:,:].sel(lat=slice(-5,5),
                                       lon=slice(210,260)).mean(dim='lat').mean(dim='lon')

nino_climo = nino_total.groupby('time.month').mean(dim='time')
nino_anom = nino_total.groupby('time.month') - nino_climo

sst = dTEMP.votemper[:,0,:,:]
sst_climo = sst.groupby('time.month').mean(dim='time')
sst_anom = sst.groupby('time.month') - sst_climo

X = sst_anom[0:600,:,:]

### Turn into categorization problem with 1's and 0's for warm and cold events

In [ ]:
yy = nino_anom[0:600]
l_warm = ( yy > 0.5 )
l_cold = ( yy < 0.5 )
y = [1*l_warm,1*l_cold]
zz = np.transpose(np.asarray(y))

### Make a keras model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras import regularizers
from keras import initializers

In [ ]:
# Define the keras model

inputdim = 180*360
model = Sequential()
model.add(Dense(8, input_dim=inputdim,activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(2, activation='softmax'))

# Compile keras model
model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])

### Fit the model to the data

In [ ]:
foo = np.asarray(X)
XX = foo.reshape(600,inputdim)
XX[np.isnan(XX)] = 0.0

# Note: I reduced the number of epochs from 600 to 100 for testing
model.fit(XX, zz, epochs=100, batch_size=60)

### Apply Innvestigate

In [ ]:
import innvestigate
import innvestigate.utils

In [ ]:
# Strip softmax layer
model = innvestigate.utils.model_wo_softmax(model)

# Create analyzer
analyzer = innvestigate.create_analyzer("deep_taylor", model)

# Apply analyzer w.r.t. maximum activated output-neuron
a = analyzer.analyze(XX)

### Reshape to lat,lon and plot

In [ ]:
a=a.reshape((180,360))
plt.pcolormesh(a)
plt.colorbar()